In [ ]:
from dialoghelper import add_msg
import re
from fastcore.foundation import Path
def md_to_notes(path):
    "Read markdown file and create a note for each header section"
    txt = Path(path).read_text()
    parts = re.split(r'^(#{1,4}\s+.+)$', txt, flags=re.MULTILINE)
    if parts[0].strip(): add_msg(content=parts[0].strip())
    for i in range(1, len(parts), 2):
        content = parts[i] + (parts[i+1] if i+1 < len(parts) else '')
        if content.strip(): add_msg(content=content.strip())

In [ ]:
md_to_notes('./md/ch07.md')

## Chapter 7

## Constrained Optimization

In this chapter, our work in unconstrained optimization is expanded to the case of identifying optima under some set of constraints for the decision variable  $x$ . As in the previous chapter, we begin with some preliminary discussion and establish necessary and sufficient conditions for a point  $x^*$  to be a solution for a class of objective functions. Whereas before this was a fairly straightforward endeavor, here we feel the need to illustrate the problem at hand with a few examples for clarity. After this motivation, we lay out the so-called Karush-Kuhn-Tucker (KKT) conditions for constrained optimization and provide a derivation of the first order necessary conditions. In so doing, we also develop some understanding of the associated Lagrange multipliers of the problem.

As was the case for unconstrained optimization, we again emphasize the attractive properties of convex functions.

With the basics in hand, we look at two broad classes of constrained optimization problems: Quadratic Programming with Linear Constraints (QPLC) and Linear Programming with Linear Constraints (LPLC). In each case, we establish standard forms of the problem and give some motivation as to how they might solved algorithmically by utilizing the KKT conditions of the particular problem. In the case of LPLC, we also lightly develop the concept of the dual.

We conclude with examples in QPLC and LPLC, ranging from a portfolio optimization problem identifying the market portfolio as the minimum variance portfolio with  $\beta = 1$  to a regression technique called quantile regression.

### 7.1 Preliminaries

For functions  $f: \mathbb{R}^N \to \mathbb{R}$  and  $c_i: \mathbb{R}^N \to \mathbb{R}$ , constrained optimization problems are given by

$$\begin{array}{ll}\min_x & f(x) \\ & c_i(x) = 0 \quad i \in \mathcal{E} \\ & c_i(x) \ge 0 \quad i \in \mathcal{I},\end{array}\tag{7.1}$$

where  $\mathcal{E}$  and  $\mathcal{I}$  denote sets of equality and inequality indices, respectively. We say that a point  $\tilde{x}$  is a *feasible point*, or simply *feasible*, if it satisfies  $c_i(\tilde{x}) = 0$  for  $i \in \mathcal{E}$  and  $c_i(\tilde{x}) \ge 0$  for  $i \in \mathcal{I}$ . The *feasible region* is the set of all feasible points, denoted

$$\chi = \{x \mid x \text{ is a feasible point}\}. \tag{7.2}$$

The functions  $c_i(\cdot)$  are called the equality and inequality constraints based on membership in  $\mathcal{E}$  and  $\mathcal{I}$ , respectively. Oftentimes, in the literature, the equation set in (7.1) is stated simply as

$$\min_{x \in \chi} f(x)$$

with the feasible set,  $\chi$  defined by some set of functions  $\{c_i(\cdot)\}_{i \in \mathcal{I} \cup \mathcal{E}}$ .

To obtain solutions to (7.1), we have to develop a different set of criteria than in the unconstrained case; viz.,  $\chi$  may not contain any stationary points of  $f(\cdot)$ . Our main tool will be the so-called Karush-Kuhn-Tucker (KKT) conditions, which we motivate in several examples that follow. We will consider in turn the case of a single equality constraint and then a single inequality constraint. These examples, are not intended to be fully rigorous, but rather to illustrate the core ideas of the necessity of the KKT conditions using the math previously presented before tackling the proof of the same.

#### 7.1.1 The Case of One Equality Constraint

Consider

$$\begin{array}{c} \min_x \\ c_1(x) = 0, \end{array} f(x)$$

where clearly,  $\mathcal{E} = \{1\}$  and  $\mathcal{I} = \{\emptyset\}$ , the empty set. Assume that both  $f$  and  $c_1 \in C^1$  so that each function and its gradient are continuous. Then we have by (5.17) that

$$c_1(x + \delta) = c_1(x) + \delta' \nabla c_1(x) + o(||\delta||). \tag{7.3}$$

For a feasible  $x$ , then, up to first order, a move in the direction  $\delta$  will stay feasible if

$$c_1(x) + \delta' \nabla c_1(x) = 0.$$

But since  $x$  is feasible,  $c_1(x) = 0$ , so this reduces to requiring

$$\delta' \nabla c_1(x) = 0. \tag{7.4}$$

For  $\delta$  to be a descent direction as in (5.4), we require

$$\delta' \nabla f(x) < 0. \tag{7.5}$$

In the case that  $x$  is not a stationary point and  $\nabla f(x) \neq \lambda \nabla c_1(x)$  for some constant  $\lambda$ , we may show there exists a  $\delta$  simultaneously satisfying (7.4) and

(7.5). That is, a direction that both maintains feasibility (up to first order) and is a descent direction; viz.,

$$\delta = -\nabla f + \frac{(\nabla f' \nabla c_1) \cdot \nabla c_1}{\|\nabla c_1\|^2} \quad (7.6)$$

For this  $\delta$ , we see that

$$\begin{aligned} \delta' \nabla c_1(x) &= -\nabla f' \nabla c_1 + \frac{(\nabla f' \nabla c_1) \cdot \nabla c_1' \nabla c_1}{\|\nabla c_1\|^2} \\ &= -\nabla f' \nabla c_1 + \nabla f' \nabla c_1 \\ &= 0, \end{aligned}$$

so that  $\delta$  maintains first order feasibility. Continuing in the same fashion,

$$\begin{aligned} \delta' \nabla f(x) &= -\nabla f' \nabla f + \frac{(\nabla f' \nabla c_1) \cdot \nabla c_1' \nabla f}{\|\nabla c_1\|^2} \\ &= -\|\nabla f\|^2 + \frac{\|\nabla f' \nabla c_1\|^2}{\|\nabla c_1\|^2}. \end{aligned}$$

By Cauchy-Schwarz, we have that  $\|\nabla f' \nabla c_1\|^2 = \beta \|\nabla f\|^2 \|\nabla c_1\|^2$ , for some  $\beta \in [0, 1]$ . But since by assumption,  $\nabla f(x) \neq \lambda \nabla c_1(x)$ , the range for  $\beta$  is reduced to  $\beta \in [0, 1)$ . Hence

$$\begin{aligned} \delta' \nabla f(x) &= -\|\nabla f\|^2 + \frac{\beta \|\nabla f\|^2 \|\nabla c_1\|^2}{\|\nabla c_1\|^2} \\ &= -\|\nabla f\|^2 + \beta \|\nabla f\|^2 \\ &= (\beta - 1) \|\nabla f\|^2 \\ &< 0. \end{aligned}$$

Giving that this direction is also a descent direction.

In conclusion, so long as  $\nabla f(x) \neq \lambda \nabla c_1(x)$ , any non-stationary  $x$  may be perturbed by  $\delta$  to obtain  $x + \delta$  that is both feasible and a descent direction. We are motivated to consider an equation such as

$$\mathcal{L}(x, \lambda) = f(x) - \lambda c_1(x)$$

and require, for a necessary condition of optimality, that

$$\nabla_x \mathcal{L}(x^*, \lambda^*) = \nabla f^* - \lambda^* \nabla c_1^* = 0.$$

We will see that this condition is indeed the case, generalizing to many constraints.

![A 2D plot showing a circle centered at the origin with radius sqrt(5). The axes range from -3 to 3. Vectors representing the gradient of the objective function, $\nabla f$, and the gradient of the constraint function, $\nabla c$, are shown at four points on the circle. The vectors $\nabla f$ point radially outward, and the vectors $\nabla c$ point radially inward. At the points where the vectors are shown, $\nabla f$ and $\nabla c$ are orthogonal.](19d3d46ab06be40b6033e9669bfa7299_img.jpg)

Objective and Constraint Gradient Plots

A 2D plot showing a circle centered at the origin with radius sqrt(5). The axes range from -3 to 3. Vectors representing the gradient of the objective function, \$\nabla f\$, and the gradient of the constraint function, \$\nabla c\$, are shown at four points on the circle. The vectors \$\nabla f\$ point radially outward, and the vectors \$\nabla c\$ point radially inward. At the points where the vectors are shown, \$\nabla f\$ and \$\nabla c\$ are orthogonal.

Figure 7.1: Plot of the gradient of the objective function and constraint function for minimizing  $2x_1 + 2x_2$  on the circle of radius  $\sqrt{5}$ .

**Example 7.1.1.** Consider the following minimization problem constrained to a circle of radius  $\sqrt{5}$ :

$$\begin{array}{ll}\min_x & f(x) = 2x_1 + 2x_2 \\ & c(x) = 5 - x_1^2 - x_2^2 = 0\end{array}$$

Notice that (by careful construction) this problem exhibits symmetry in the two variables in question. As such, the problem reduces to

$$\begin{array}{ll}\min_x & f(x) = 4x \\ & c(x) = 5 - 2x^2 = 0\end{array}$$

where now  $x \in \mathbb{R}$ . A solution is clearly determined from the constraints, which determine that  $x = \pm\sqrt{\frac{5}{2}}$ . This gives four possible points where the minimum may occur, and we find that  $x^* = -\sqrt{\frac{5}{2}}$  achieves the minimum value of  $f$  on the circle.

Now, the gradient of the constraint function is given by

$$\nabla c(x) = \begin{pmatrix} -2x_1 \\ -2x_2 \end{pmatrix}.$$

Plotting the gradient of  $f$ ,  $\nabla f \equiv (2, 2)'$ , we see in Figure 7.1.1, where the magnitude of the vectors have each been normalized, that at the minimum,  $x^*$ , the gradients of  $f$  and  $c$  are parallel. Elsewhere at the maximum  $(-x^*)$ , we have that the angle between  $\nabla f$  and  $\nabla c$  is  $\pi$ .

#### 7.1.2 The Case of One Inequality Constraint

We next look at the case of one inequality constraint,

$$\begin{array}{ll} \min_x & f(x) \\ & c_1(x) \ge 0, \end{array}$$

with,  $\mathcal{E} = \{\emptyset\}$  and  $\mathcal{I} = \{1\}$ . Again, assuming that  $f$  and  $c_1 \in C^1$ , we have in a manner similar to the above that first order requirements for a feasible descent direction (for nonstationary point,  $x$ ) are

$$\begin{array}{rcl} c_1(x) + \delta' \nabla c_1(x) & \ge & 0 \\ \delta' \nabla f(x) & < & 0. \end{array}$$

We consider two cases:  $x$  on the interior of the disc and  $x$  on the boundary.

When on the interior, if not at a stationary point, we may find a  $\gamma$  small enough so that  $\delta = -\gamma \nabla f(x)$  remains feasible. Clearly, as well,  $\delta$  is a descent direction.

On the boundary,  $c_1(x) = 0$ , so we require

$$\begin{array}{rcl} \delta' \nabla c_1(x) & \ge & 0 \\ \delta' \nabla f(x) & < & 0. \end{array}$$

By Cauchy-Schwarz, we have then that

$$\begin{array}{rcl} \delta' \nabla c_1 & = & \cos \theta_1 ||\delta|| \cdot ||\nabla c_1||, \quad \theta_1 \in \left[-\frac{\pi}{2}, \frac{\pi}{2}\right] \\ \delta' \nabla f & = & \cos \theta_2 ||\delta|| \cdot ||\nabla f||, \quad \theta_2 \in \left(\frac{\pi}{2}, \frac{3\pi}{2}\right). \end{array}$$

The set of feasible descent directions is determined by a cone constructed from the intersection of the regions determined above. One may convince themselves from this that if  $\nabla f$  and  $\nabla c_1$  are not parallel, there will always be a cone of feasible directions reducing  $f$ .

We again see motivation to consider  $\mathcal{L}(x, \lambda) = f(x) - \lambda c_1(x)$  and the necessity of requiring  $\nabla_x \mathcal{L}(x^*, \lambda^*) = 0$  for  $x^*$  a minimizer of  $f$  on the feasible region. In this case, we have an additional requirement that  $\lambda^* \ge 0$ , with positivity of  $\lambda^*$  if  $x^*$  is on the boundary. On the interior, optimality occurs at a stationary point, and hence  $\lambda^* = 0$  in this simple example.

These requirements are called *strict complementarity*. In particular, the product,  $\lambda^* c_1^* = 0$ , but either  $\lambda^* > 0$  and  $c_1 = 0$  or  $\lambda^* = 0$  and  $c_1 > 0$ . The pair are never simultaneously zero unless  $f$  has a stationary point on the boundary.

#### 7.1.3 First and Second Order Conditions

As in the unconstrained case, we obtain first and second order necessary and sufficient conditions for optimality for the constrained problem (7.1).

Before proceeding, we say that  $x$  is a *regular point* of (7.1) if

$$\{\nabla c_i(x) | i \in \mathcal{A}(x)\} \tag{7.7}$$

is linearly independent, where  $\mathcal{A}(x)$  is the *active set* at  $x$  given by

$$\mathcal{A}(x) = \mathcal{E} \cup \{i \in \mathcal{I} | c_i(x) = 0\}, \tag{7.8}$$

or the set of indices where equality obtains in the constraint set. We further say that an inequality constraint,  $c_i$  is *active* at  $x$  if  $c_i(x) = 0$ .

Finally, for a feasible point  $x$ , and motivated by the linear approximations used in the examples above, we define the *set of linearized feasible directions*,  $\mathcal{F}(x)$  by

$$\mathcal{F}(x) = \left\{d \middle| \begin{array}{l} d'\nabla c_i(x) = 0, \quad i \in \mathcal{E} \\ d'\nabla c_i(x) \ge 0, \quad i \in \mathcal{A}(x) \cap \mathcal{I} \end{array} \right\} \tag{7.9}$$

With these definitions, the necessary and sufficient Karush-Kuhn-Tucker conditions for constrained optimization problems given by (7.1) may be stated. In each case that follows, we define the *Lagrangian function* by

$$\mathcal{L}(x, \lambda) = f(x) - \sum_{i \in \mathcal{E} \cup \mathcal{I}} \lambda_i c_i(x). \tag{7.10}$$

Each component,  $\lambda_i$ , of the vector  $\lambda$  (whose size is equal to the total number of constraints) is called a *Lagrange multiplier*.

**First Order Necessary Condition** For  $x^*$  a regular point and local solution to the constrained optimization problem (7.1) with  $f$  and each  $c_i$  in  $C^1$ , there exists a  $\lambda^*$  such that the following conditions hold:

$$\nabla_x \mathcal{L}(x^*, \lambda^*) = 0 \tag{7.11a}$$

$$x^* \text{ is feasible} \tag{7.11b}$$

$$\lambda_i^* \ge 0, \text{ for all } i \in \mathcal{I} \tag{7.11c}$$

$$\lambda_i^* c_i(x^*) = 0, \text{ for all } i \in \mathcal{E} \cup \mathcal{I}. \tag{7.11d}$$

Collectively the equations in (7.11) are known as the Karush-Kuhn-Tucker conditions, or KKT, while the final conditions are called complementarity conditions.

**Second Order Necessary Condition** To state the second order necessary condition here, we must first define the *critical cone*. For  $\mathcal{F}(x^*)$  the set of feasible linear directions as in (7.9) and  $(x^*, \lambda^*)$  a pair satisfying the KKT conditions (7.11), the *critical cone*,  $\mathcal{C}(x^*, \lambda^*)$ , is defined by

$$\mathcal{C}(x^*, \lambda^*) = \{d \in \mathcal{F}(x^*) | d'\nabla c_i^* = 0, i \in \mathcal{A}(x^*) \cap \mathcal{I}, \text{ with } \lambda_i^* > 0\}. \tag{7.12}$$

By complementarity, we see that the critical cone contains the set of directions that maintain linear feasibility and also retain up to first order the same active constraint set. Further it follows from complementarity of KKT and the definition here that if  $d\in\mathcal{C}(x^*,\lambda^*)$ , then

$$\lambda_i^*d'\nabla c_i^*=0$$

for all  $i\in\mathcal{E}\cup\mathcal{I}$ .

We have that if  $d\in\mathcal{C}(x^*,\lambda^*)$ ,

$$d'\nabla f^*=\sum_{i\in\mathcal{E}\cup\mathcal{I}}\lambda_i^*d'\nabla c_i^*=0.$$
 (7.13)

As a result, the critical cone contains the set of all feasible linear directions for which the first order necessary conditions are not clear with regards to an increase or decrease in  $f$ .

For  $x^*$  a regular point and local solution to the constrained optimization problem (7.1) with  $f$  and each  $c_i$  in  $\mathbb{C}^2$ , and  $\lambda^*$  a solution to (7.11), then

$$d'\nabla_{xx}^2\mathcal{L}(x^*,\lambda^*)d\ge 0$$
 (7.14)

for all  $d\in\mathcal{C}(x^*,\lambda^*)$ .

**Second Order Sufficient Condition** For  $f$  and each  $c_i$  in  $\mathbb{C}^2$ , if  $x^*$  is feasible and  $(x^*,\lambda^*)$  satisfy the KKT conditions and

$$d'\nabla_{xx}^2\mathcal{L}(x^*,\lambda^*)d>0$$
 (7.15)

for all  $d\in\mathcal{C}(x^*,\lambda^*)$ , then  $x^*$  is a strict local minimizer of (7.1).

#### 7.1.4 Mathematical Derivation for First Order Conditions

In much of the preceding section, we gave motivating examples for the first order necessary conditions encapsulated in the KKT conditions just cited. Here we give a formal treatment using the inverse mapping theorem.

As before, we begin by considering the case of having only equality constraints. Define the map  $F:\mathbb{R}^N\rightarrow\mathbb{R}^M$  by

$$F(x)=\begin{pmatrix} c_1(x) \\ \vdots \\ c_m(x) \end{pmatrix}$$

where  $\mathcal{E}=\{1,\dots,m\}$ . If  $x^*$  is optimal, and the Jacobian of  $F$  is locally invertible at  $x^*$  (equivalently,  $x^*$  is a regular point), then there exists a smooth map  $G:\mathbb{R}^N\rightarrow\mathbb{R}^N$  such that  $G$  satisfies

$$x^*=G(z^*)$$

for some  $z^*$ , with  $\nabla G(z^*)$  invertible. Further, the coordinates  $z$  may be chosen such that the composite constraint functions,  $\tilde{c}_i(z) = c_i(G(z))$  satisfy

$$\tilde{c}_i(z) = z_i.$$

In  $z$ , then, the original constraints become simply  $z_i = 0$  for  $i \in \mathcal{E}$ .

As a result, the composite objective function,  $\tilde{f}(z) = f(G(z))$  must satisfy the first order necessary condition

$$\frac{\partial \tilde{f}}{\partial z_i}(z^*) = 0$$

for  $i \in \{m+1, \dots, N\}$ .

Defining

$$\lambda_i^* = \frac{\partial \tilde{f}}{\partial z_i}(z^*)$$

for all indexes, then, we must have

$$\frac{\partial \tilde{f}}{\partial z_i}(z^*) - \lambda_j^* \frac{\partial \tilde{c}_j}{\partial z_i}(z^*) = 0$$

over all indexes  $i$ , and for any  $j \in \mathcal{E}$  since  $\tilde{c}_j(z) \equiv z_j$ . This may be succinctly written as

$$\nabla \tilde{f}(z^*) - \sum_{i \in \mathcal{E}} \lambda_i^* \nabla \tilde{c}_i(z^*) = 0. \quad (7.16)$$

Finally, since  $x^* = G(z^*)$ , we may write the above in the original coordinates as

$$\nabla f(x^*) - \sum_{i \in \mathcal{E}} \lambda_i^* \nabla c_i(x^*) = 0. \quad (7.17)$$

Hence we have shown that if  $x^*$  is a regular point, a necessary condition for  $x^*$  to be optimal is that there exists a  $\lambda^*$  such that the gradient of the Lagrangian for the problem (as previously defined) vanishes for both gradients in  $x$  and  $\lambda$  at  $(x^*, \lambda^*)$ ; viz.,

$$\nabla_x \mathcal{L}(x^*, \lambda^*) = 0 \quad (7.18)$$

$$\nabla_\lambda \mathcal{L}(x^*, \lambda^*) = 0, \quad (7.19)$$

the second equation being a concise way to denote the feasibility of  $x^*$ . Notice that this is exactly the first order necessary condition previously stated.

The generalization to inequality constraints is handled similarly. As before, we may (smoothly) change to an easier coordinate system, but now accounting for the active set at  $x^*$ ,  $\mathcal{A}(x^*)$ , rather than just the equality set,  $\mathcal{E}$ . Again, so long as the active constraints are linearly independent at  $x^*$ , there exists a  $G$  as before satisfying

$$\tilde{c}_i(z) = c(G(x)) = z_i$$

for  $i \in \mathcal{A}(x^*)$ .

In this coordinate system, we seek to minimize  $\tilde{f}(z) = f(G(z))$  subject to

$$z_i = 0, i \in \mathcal{E}$$
$$z_i \ge 0, i \in \mathcal{A}(x^*) \cap \mathcal{I},$$

with  $z_i^* = 0$  for  $i \in \mathcal{A}(x^*)$  by construction. The first order necessary conditions of optimality of  $z^*$  become, then,

$$\frac{\partial \tilde{f}}{\partial z_i}(z^*) = 0, i \notin \mathcal{A}(x^*)$$
$$\frac{\partial \tilde{f}}{\partial z_i}(z^*) \ge 0, i \in \mathcal{A}(x^*) \cap \mathcal{I}.$$

Defining

$$\lambda_i^* = \frac{\partial \tilde{f}}{\partial z_i}(z^*), i \in \mathcal{A}(x^*)$$
$$\lambda_i^* = 0, i \notin \mathcal{A}(x^*),$$
(7.20)

we have that

$$\frac{\partial \tilde{f}}{\partial z_i}(z^*) - \lambda_j^* \frac{\partial \tilde{c}_j}{\partial z_i}(z^*) = 0.$$

This may be seen by considering the various partitions of the index,  $i$ . In any partition, however, we note that the term  $\lambda_j^* \frac{\partial \tilde{c}_j}{\partial z_i}(z^*)$  is only nonzero when  $j = i$  by our choice of coordinate system and definition of  $\lambda$  above.

For  $i \notin \mathcal{E} \cup \mathcal{I}$ ,  $\frac{\partial \tilde{f}}{\partial z_i}(z^*)$  vanishes and there are no  $j$ 's that can equal  $i$  in  $\lambda_j^* \frac{\partial \tilde{c}_j}{\partial z_i}$  on this set of indexes. Hence these terms are all zero as well.

When  $i \in \mathcal{A}(x^*)$ , we have that

$$\lambda_i^* \frac{\partial \tilde{c}_i}{\partial z_i}(z^*) = \frac{\partial \tilde{f}}{\partial z_i}(z^*)$$

and hence the sum in question is zero.

Finally, those indexes in the inequality set which are inactive must have both vanishing  $\frac{\partial \tilde{f}}{\partial z_i}(z^*)$  and  $\lambda_i^*$ .

As before, we may write these results concisely as

$$\nabla \tilde{f}(z^*) - \sum_{i \in \mathcal{E} \cup \mathcal{I}} \lambda_i^* \nabla \tilde{c}_i(z^*) = 0,$$
(7.21)

or, in the original coordinate system

$$\nabla f(x^*) - \sum_{i \in \mathcal{E} \cup \mathcal{I}} \lambda_i^* \nabla c_i(x^*) = 0.$$
(7.22)

The result may be summarized by saying that in the case of both equality and inequality constraints, a necessary condition for a feasible regular point,  $x^*$ , to be optimal is that there exists  $\lambda^*$  satisfying

$$\nabla_x \mathcal{L}(x^*, \lambda^*) = 0$$

with those  $\lambda_i^*$  associated with inequality constraints being positive when inactive and zero when active; that is, strict complementarity holds across all constraints. As before, this result coincides with our previous summary of the KKT conditions.

Finally, we notice that (7.20) indicates an interpretation of the Lagrange multipliers satisfying the KKT conditions. From the equations there is some sense that  $\lambda_i^*$  is equivalent to the sensitivity of the objective function to small changes in the  $i$ th constraint; viz., for the inactive constraints, we would expect the impact to be zero since the constraints are nonbinding.

While this is clear in the alternative coordinate system produced, we may also prove this result with a more elementary derivation. Focusing on the standard optimization problem (7.1), we let  $x^*$  and  $\lambda^*$  satisfy the associated first order KKT conditions. Next we consider the case of changing the  $j$ th constraint in the active set at  $x^*$  from  $c_j(x)=0$  to  $c_j(x)=\delta$ .

The result of this change would be to obtain a new optimal solution for the corresponding problem which we may write as  $x^*+\Delta x^*$ . The approximate change in objective function values is given by

$$f(x^*+\Delta x^*)-f(x^*)\approx\nabla f(x^*)'\Delta x^*,$$

which we may make exact with notation as in (5.17), for example. By the first order KKT conditions, we have that

$$\nabla f(x^*)'\Delta x^*=\sum_{i\in\mathcal{A}(x^*)}\lambda_i^*\nabla c_i(x^*)'\Delta x^*.$$

We may also write a linear approximation of the constraint functions as

$$c_i(x^*+\Delta x^*)-c_i(x^*)\approx\nabla c_i(x^*)'\Delta x^*,$$

for each  $i$  in the active set. For each  $i\neq j$ , however, we have that the left hand side of this equation is zero. For  $i=j$ , we are left with

$$\delta=\nabla c_j(x^*)'\Delta x^*.$$

This gives immediately that

$$\nabla f(x^*)'\Delta x^*=\lambda_j^*\delta,$$

and hence

$$\frac{f(x^*+\Delta x^*)-f(x^*)}{\delta}\approx\lambda_j^*.$$

Taking the limit as  $\delta\rightarrow 0$  confirms the result.

We note that, smoothness requirements for both  $f$  and  $c$ . are established based upon the Taylor approximations used. In particular, the above proof required that these functions simply be differentiable at  $x^*$ .

### 7.2 Convex Functions and KKT

The first order KKT conditions on  $(x^*, \lambda^*)$  are sufficient in the case that  $f$  is convex and each  $c_i$  is concave (i.e.,  $-c_i$  is convex). In other words, the satisfaction of the KKT conditions in this case ensures a solution to the constrained problem.

Before proceeding, we note that the set

$$K = \{x | c_i(x) \ge k_i\} \quad (7.23)$$

is convex when each  $c_i$  is concave and leave the proof to the reader. With respect to the constrained optimization problem we are considering, this implies that the feasible set is convex.

Assume next that  $(x^*, \lambda^*)$  satisfy the first order KKT conditions with convex  $f$  and concave  $c_i$ . For any point,  $x_0$ , we have that

$$f(x_0) \ge f(x_0) - \sum_i \lambda_i^* c_i(x_0). \quad (7.24)$$

And by convexity of  $f$ ,

$$f(x_0) \ge f(x^*) + \nabla f(x^*)'(x_0 - x^*). \quad (7.25)$$

Similarly, by concavity of  $c_i$ ,

$$-c_i(x_0) \ge -c_i(x^*) - \nabla c_i(x^*)'(x_0 - x^*). \quad (7.26)$$

By (7.24), (7.25), and (7.26), then,

$$\begin{aligned} f(x_0) &\ge f(x_0) - \sum_i \lambda_i^* c_i(x_0) \\ &\ge f(x^*) + \nabla f(x^*)'(x_0 - x^*) - \sum_i \lambda_i^* c_i(x_0) \\ &\ge f(x^*) + \nabla f(x^*)'(x_0 - x^*) - \sum_i (c_i(x^*) + \nabla c_i(x^*)'(x_0 - x^*)) \\ &\ge f(x^*) + \left( \nabla f(x^*) - \sum_i \lambda_i^* \nabla c_i(x_0) \right)' (x_0 - x^*) - \sum_i \lambda_i^* c_i(x^*). \end{aligned}$$

At  $x^*$ , the KKT conditions give that the gradient of the Lagrangian is zero so that

$$\nabla f(x^*) - \sum_i \lambda_i^* \nabla c_i(x^*) = 0$$

and further, by strict complementarity,

$$\sum_i \lambda_i^* c_i(x^*) = 0.$$

These observations in concert with the last line of the inequality above yield that  $f(x_0) \ge f(x^*)$  and show that  $f(x^*)$  is the minimum value obtained on the constrained set.

### 7.3 Quadratic Programming with Linear Constraints

Of particular importance will be the case of a quadratic objective function with linear constraints. Here we consider the so-called Quadratic Problem with Linear Constraints (QPLC) given by

$$\min_x q(x) = \frac{1}{2} x' Q x - r' x$$
$$A x = b. \tag{7.27}$$

The general case of linear inequality constraints is also a QPLC.

We assume that  $A \in \mathbb{R}^{M \times N}$  is full rank and, for now, that  $Q$  is simply symmetric. We will tighten our assumptions on  $Q$  below. Relating this new formulation to the notation above, we may write

$$A = \begin{pmatrix} - & a_1 & - \\ - & a_2 & - \\ \vdots & & \vdots \\ - & a_M & - \end{pmatrix}$$

as

$$c_i(x) = a_i \cdot x - b_i.$$

The gradient of  $q$  is found directly to be

$$\nabla q(x) = Qx - r$$

since  $Q$  is symmetric, and the gradient of each  $c_i$  is exactly  $a'_i$ . As a result, we may state the KKT condition on the gradient of the Lagrangian as

$$Qx^* - r - \sum_i \lambda_i^* a'_i = 0. \tag{7.28}$$

Looking at the final sum, we see that

$$\begin{aligned} \sum_i \lambda_i^* a'_i &= \lambda_1^* \begin{pmatrix} | \\ a_1 \\ | \end{pmatrix} + \cdots + \lambda_M^* \begin{pmatrix} | \\ a_M \\ | \end{pmatrix} \\ &= A' \lambda. \end{aligned}$$

As a result, (7.28) may be rewritten as

$$Qx^* - r - A' \lambda^* = 0. \tag{7.29}$$

The only remaining condition, feasibility, is given by  $Ax^* = b$ .

In all, then, the KKT conditions for this problem may be summarized as a system of linear equations

$$\begin{pmatrix} Q & -A' \\ A & 0 \end{pmatrix} \begin{pmatrix} x^* \\ \lambda^* \end{pmatrix} = \begin{pmatrix} r \\ b \end{pmatrix}. \tag{7.30}$$

In the specific case of  $Q \succ 0$ , we know from our previous work that the system must have a unique solution. Here we prove a uniqueness result for a less strict condition on  $Q$ .

For  $K$  the matrix given in (7.30) and  $Q$  symmetric and satisfying  $Z'QZ \succ 0$  for  $Z$  the matrix made up of the columns of the basis of the null space of  $A$ ,  $\text{null}(A)$ , there exists a unique solution to (7.27).

*Proof.* We prove that  $K$  is nonsingular. Suppose  $(v, w)'$  satisfies

$$K \begin{pmatrix} v \\ w \end{pmatrix} = 0$$

for some  $v$  and  $w$  not identically zero. Then we must have  $Av = 0$  and  $v \in \text{null}(A)$ . Now, the inner product of  $(v, w)'$  with respect to  $K$  gives

$$\begin{aligned}(v' w') \begin{pmatrix} Q & -A' \\ A & 0 \end{pmatrix} \begin{pmatrix} v \\ w \end{pmatrix} &= (v' w') \begin{pmatrix} Qv - A'w \\ Av \end{pmatrix} \\ &= v'Qv - v'A w + w'A v \\ &= 0.\end{aligned}$$

Since  $v$  is in the null space of  $A$ , this reduces to

$$v'Qv = 0.$$

Continuing, we may write  $v = Zu$  for some  $u$  since  $Z$  spans  $\text{null}(A)$ . This gives, then, that

$$u'Z'QZu = 0,$$

contradicting the positive definiteness of  $Z'QZ$ . Therefore  $u \equiv 0$  and so is  $v$ .

Next we show that  $w$  must be zero as well. From the first set of equations in the system, we have that

$$Qv - A'w = 0,$$

which reduces to  $A'w = 0$  as  $v \equiv 0$ . Finally, since  $A$  has full rank,  $A'w = 0$  implies  $w$  must be zero as well.

We conclude that  $K$  is nonsingular as desired.

□

While it is interesting that the process shown above yields the solution to the constrained QP problem is exactly that of a linear system of equations in the extended variable set including the Lagrange multipliers, it is also a sketch that leads to a more general class of algorithms called interior point methods. We give a first indication of such an implementation in the exercises; viz., how might the above be generalized to non-quadratic objective functions and how might an iteration scheme be developed?

### 7.4 Linear Programming with Linear Constraints

Another indispensable framework is the Linear Program with Linear Constraints (LPLC), or simply, Linear Programming, the standard form of which is given by

$$\begin{array}{ll}\min_x & c'x \\ & Ax = b \\ & x \ge 0.\end{array}\tag{7.31}$$

The formulation in (7.31) is more flexible than it might first appear. For example, inequality constraints such as

$$Cx \ge d$$

may be described with the use of so-called slack variables,  $s$ , as

$$\begin{array}{ll}Cx - s & = d \\ s & \ge 0.\end{array}$$

The nonnegativity constraint on  $x$  is also less restrictive than might otherwise be supposed. The general case of

$$\begin{array}{ll}\min_x & c'x \\ & Ax = b \\ & Cx \ge d\end{array}\tag{7.32}$$

may be written in the standard form by writing  $x$  as a difference of its positive and negative parts,

$$x = x_+ - x_-$$

with each of  $x_+$  and  $x_-$  being nonnegative. The objective function in this case becomes

$$\begin{pmatrix} c \\ -c \\ 0 \end{pmatrix}' \begin{pmatrix} x_+ \\ x_- \\ s \end{pmatrix}$$

where we have anticipated the need for slack variables. Completing the formulation of (7.32) in standard form is left as an exercise.

Without loss of generality, then, we may focus on problems of the type (7.31). The Lagrangian for this problem is

$$\mathcal{L}(x, \lambda, \eta) = c'x - \lambda'(Ax - b) - \eta'x,$$

and the KKT conditions are

$$\nabla_x \mathcal{L}(x^*, \lambda^*, \eta^*) = c - A'\lambda^* - \eta^* = 0 \tag{7.33a}$$

$$Ax^* = b \tag{7.33b}$$

$$x^* \ge 0 \tag{7.33c}$$

$$\eta^* \ge 0 \tag{7.33d}$$

$$\eta^{*'} x^* = 0. \tag{7.33e}$$

Notice that the complementarity condition has been replaced by  $\eta^{*\prime}x^*=0$ , but that these conditions are equivalent due to the non-negativity of  $x^*$  and  $\eta^*$ .

We next show directly that the KKT conditions in (7.33) are sufficient for an optimal solution of (7.32). For any feasible  $\tilde{x}$ , we have that the value in the objective function is

$$\begin{aligned}c'\tilde{x} &= (A'\lambda^* + \eta^*)'\tilde{x} \\&= \lambda^{*\prime}A\tilde{x} + \eta^{*\prime}\tilde{x} \\&= \lambda^{*\prime}b + \eta^{*\prime}\tilde{x} \\&\ge \lambda^{*\prime}b.\end{aligned}$$

Now the objective function at  $x^*$  is related to this final linear equation in the Lagrange multiplier,  $\lambda^*$  vis a vis the same steps as above, but noting that  $\eta^{*\prime}\tilde{x}=0$ . By doing so, we see that

$$c'x^* = \lambda^{*\prime}b. \quad (7.34)$$

Putting this into the last line of the above, we conclude that  $c'\tilde{x} \ge c'x^*$  and the KKT conditions are sufficient for  $x^*$  to be a solution to the original problem.

The relationship in (7.34) is significant in its own right. In fact, with this equation and an analysis of the KKT conditions in (7.33), we may construct the so-called dual problem and note the Strong Duality Theorem of linear programming. In what follows, we will refer to our original linear programming problem as the primal.

Suppose that  $(x^*, \lambda^*, \eta^*)$  is a solution to the first order KKT conditions in (7.33). The condition

$$c - A'\lambda^* - \eta^* = 0$$

is indicative of a linear inequality in  $\lambda$ . With this intuition along with (7.34), we write the dual problem of (7.31) as

$$\begin{array}{ll}\max_{\lambda} & b'\lambda \\& A'\lambda \le c.\end{array} \quad (7.35)$$

Verifying that the KKT conditions of (7.35) coincide with those of the primal confirms this choice. We have

$$\begin{aligned}\nabla_{\lambda}\tilde{\mathcal{L}}(x^*, \lambda^*, \eta^*) &= -b + Ax^* = 0 \\A'\lambda^* &\le c \\x^* &\ge 0 \\x_i^*(a_i x^* - c) &= 0.\end{aligned}$$

Substituting  $\eta^* = c - A'\lambda^*$  and rearranging some terms gives

$$\begin{aligned}Ax^* &= b \\ \eta^* &\ge 0 \\ x^* &\ge 0 \\ \eta^{*\prime}x^* &= 0,\end{aligned}$$

which is exactly the KKT conditions determined for the original problem. We have already shown, too, that the objective function values at optimal solutions are equal for both the primal and dual. The variables  $(\lambda, \eta)$  are often referred to as the dual variables for (7.31).

We collect these results in the following theorem, the second portion of which requires some further development, but is readily available from the above.

**Theorem 7.4.1** (Strong Duality).

If the primal problem (7.31) affords a finite solution, then so does the dual (7.35), and their objective functions are equal. Conversely, if the dual has a finite solution so does the primal.

If the primal (dual) is unbounded, then the dual (primal) is infeasible.

As a last point of discussion in this introductory exposition of Linear Programming, and in a manner similar to that shown for Quadratic Programming with Linear Constraints above, we motivate an algorithmic technique for solving (7.31). These algorithms are referred to as primal-dual interior point methods, and as we shall soon see, what follows is mainly a restatement of results we have already obtained. The updating technique we indicate should also feel familiar.

We begin by writing the primal KKT conditions in a functional form by first defining

$$F(x, \lambda, \eta) = \begin{pmatrix} A'\lambda + \eta - c \\ Ax - b \\ X\text{Se} \end{pmatrix} \quad (7.36)$$

where  $X = \text{diag}(x_1, \dots, x_N)$  and  $S = \text{diag}(\eta_1, \dots, \eta_N)$  and  $e$  is a vector of ones. We may concisely state the conditions now as

$$\begin{aligned} F &= 0 \\ \eta'x &= 0. \end{aligned}$$

Interior point primal-dual methods generate a sequence  $(x^k, \lambda^k, \eta^k)$  satisfying  $\eta^k > 0$  and  $x^k > 0$  (hence the qualifier ‘interior’) and approximately solving  $F^k = 0$ .

Given  $(x^k, \lambda^k, \eta^k)$ , we would like to construct an update  $(\Delta x, \Delta\lambda, \Delta\eta)$  such that  $F^{k+1} = 0$ . As we have seen with other nonlinear functions, especially when considering Newton’s method, we may approximate the nearly linear  $F$  by way of its Jacobian. We have

$$\nabla F(x, \lambda, \eta) = \begin{pmatrix} 0 & A' & I \\ A & 0 & 0 \\ S & 0 & X \end{pmatrix}. \quad (7.37)$$

To solve  $F^{k+1}$  up to first order, we see that the updates  $(\Delta x, \Delta\lambda, \Delta\eta)$  must satisfy

$$\nabla F^k \begin{pmatrix} x^{k+1} \\ \lambda^{k+1} \\ \eta^{k+1} \end{pmatrix} = \begin{pmatrix} c \\ b \\ 0 \end{pmatrix}.$$

Or,

$$\nabla F^k \begin{pmatrix} x^k \\ \lambda^k \\ \eta^k \end{pmatrix} + \nabla F^k \begin{pmatrix} \Delta x \\ \Delta \lambda \\ \Delta \eta \end{pmatrix} = \begin{pmatrix} c \\ b \\ 0 \end{pmatrix}.$$

Which gives

$$\nabla F^k \begin{pmatrix} \Delta x \\ \Delta \lambda \\ \Delta \eta \end{pmatrix} = \begin{pmatrix} c - A'\lambda^k - \eta^k \\ b - Ax^k \\ -S^k x^k - X^k \eta^k \end{pmatrix},$$

with updates now available by solving the resulting linear system.

As usual, the choice of how much of a Newton step to take is a critical refinement of the process, but a fairly accurate representation of the algorithm is obtained from the above and determining a choice of update length  $\alpha$  so that

$$(x^{k+1}, \lambda^{k+1}, \eta^{k+1}) = (x^k, \lambda^k, \eta^k) + \alpha(\Delta x, \Delta \lambda, \Delta \eta). \quad (7.38)$$

We have omitted some important details, however. In particular, we have not dealt with the question of remaining interior; viz., we have not handled  $x^{k+1} = 0$  and  $s^{k+1} = 0$  in our treatment. The full algorithm (along with its convergence behavior) is beyond the scope of the text, unfortunately, but the interested reader should be well equipped to engage such material from this point.

### 7.5 Constrained Optimization Examples

As in the unconstrained case, several examples become accessible having identified necessary and sufficient conditions for optimality. Here we establish constrained optimization problems with analytic solutions identifying the eigenvalues of a positive definite matrix, constructing prediction intervals for generalized least squares problems, relating minimum variance portfolios with  $\beta = 1$  to the market portfolio, and establishing quantile regression as a Linear Programming problem.

**Example 7.5.1.** The eigenvalues of a covariance matrix,  $\Sigma$  may be determined via constrained optimization. Consider

$$\min_x x'\Sigma x \\ ||x||^2 = 1.$$

The Lagrangian is

$$\mathcal{L}(x, \lambda) = x'\Sigma x - \lambda (||x||^2 - 1)$$

which has gradient

$$\nabla_x \mathcal{L}(x, \lambda) = 2\Sigma x - 2\lambda x.$$

We require by (7.11) that  $\nabla_x \mathcal{L}(x^*, \lambda^*) = 0$ , giving

$$\Sigma x^* = \lambda^* x^*.$$

Hence  $x^*$  is an eigenvector with associated eigenvalue  $\lambda^*$ . Let this first eigenvector, eigenvalue pair be denoted  $(v_1, \gamma_1)$ .

The next smallest eigenvalue may be found in a similar fashion. Namely by solving

$$\begin{aligned}\min_x & x'\Sigma x \\ & ||x||^2 = 1 \\ & v_1'x = 0.\end{aligned}$$

That is, find the eigenvector with smallest eigenvalue perpendicular to  $v_1$ . The first portion of this sentence needs to be validated; i.e., we are not sure that the solution here is in fact an eigenvector. We proceed with the Lagrangian as before, with

$$\mathcal{L}(x, \lambda) = x'\Sigma x - \lambda_1 (||x||^2 - 1) - \lambda_2(v_1'x).$$

The gradient is then

$$\nabla_x \mathcal{L}(x, \lambda) = 2\Sigma x - 2\lambda_1 x - \lambda_2 v_1,$$

and a pair,  $(x^*, \lambda^*)$  satisfying KKT gives  $\nabla_x \mathcal{L}(x, \lambda) = 0$ . Premultiplying the gradient of the Lagrangian by  $v_1$  and dividing through by 2, we have

$$v_1'\Sigma x^* - \lambda_1^* v_1' x^* - \frac{1}{2}\lambda_2^* ||v_1||^2 = 0.$$

By feasibility of the current problem,  $v_1'x^* = 0$ . Feasibility in the preceding problem gave  $||v_1||^2 = 1$  as well. The first order condition on  $\mathcal{L}$  gives that  $\lambda_2^* = 0$ :

$$\begin{aligned}v_1'\Sigma x^* - \frac{1}{2}\lambda_2^* &= 0 \\ \gamma_1 v_1' x^* - \frac{1}{2}\lambda_2^* &= 0 \\ \frac{1}{2}\lambda_2^* &= 0\end{aligned}$$

Hence we have

$$\Sigma x^* - \lambda_1^* x^* = 0$$

as before, and  $x^*$  is an eigenvector with associated eigenvalue  $\lambda_1^*$ . Necessarily,  $\lambda_1^* \ge \gamma_1$ .

The procedure may be continued to find all of the eigenvalues of  $\Sigma$ .

We next return to an issue not presented in our original treatment of Generalized Least Squares. Namely, given a new observation,  $X_{N+1}$ , what is the best predictor  $\hat{y}_{N+1}$ ?

**Example 7.5.2.** Consider the GLS assumptions as before,

$$Y = X\beta + \epsilon$$

with  $Cov(\epsilon) = V$  for covariance matrix  $V \in \mathbb{R}^{N \times N}$ , and estimated  $\hat{\beta}$  given by (4.49):

$$\hat{\beta} = (X'V^{-1}X)^{-1}X'V^{-1}Y.$$

Let  $X_{N+1}$  be a new observation, and let

$$\hat{y}_{N+1} = w'Y$$

be a linear predictor for some  $w$ . To ensure this estimator is unbiased, we require

$$\mathbb{E}(\hat{y}_{N+1} - y_{N+1}) = 0,$$

which gives

$$\begin{aligned}\mathbb{E}(w'Y - y_{N+1}) &= \mathbb{E}(w'(X\beta + \epsilon) - (X_{N+1}\beta + \epsilon_{N+1})) \\ &= (w'X - X_{N+1}')\beta = 0.\end{aligned}$$

Since this is true for every  $w$  that gives an unbiased estimator, we require  $w'X - X_{N+1}' = 0$ . Last, we note that the prediction error may be written

$$w'\epsilon - \epsilon_{N+1}.$$

Two issues arise: the first is that  $w$  is not uniquely determined; the second is that  $\epsilon_{N+1}$  and  $\epsilon$  are correlated under the generalized least squares assumption. Resolving the second issue is simply a matter of making an assumption about the covariance between  $\epsilon_{N+1}$  and  $\epsilon$ . We assume

$$Cov(\epsilon_{N+1}, \epsilon) = s \in \mathbb{R}^N.$$

Resolving the non-uniqueness of  $w$  may be handled by solving an optimization problem. The objective function we consider is the variance of the prediction error given by

$$\begin{aligned}Var(w'\epsilon - \epsilon_{N+1}) &= Var(w'\epsilon) - 2Cov(\epsilon_{N+1}, w'\epsilon) + Var(\epsilon_{N+1}) \\ &= w'Vw - 2w's + \sigma^2\end{aligned}$$

where  $\sigma^2$  is the variance of  $\epsilon_{N+1}$ .

Noting that we may rescale the objective function and omit the constant  $\sigma^2$ , the constrained optimization problem we consider is

$$\begin{aligned}\min_w & \frac{1}{2}w'Vw - w's \\ & X'w = X'_{N+1}\end{aligned}$$

whose Lagrangian is

$$\mathcal{L}(w, \lambda) = \frac{1}{2}w'Vw - w's - \lambda'(X'w - X'_{N+1})$$

with gradient in  $w$

$$\nabla_w \mathcal{L}(w, \lambda) = Vw - s - X\lambda$$

which, as before, we set equal to zero.

Coupling this statement about the gradient of  $\mathcal{L}$  with the constraint  $X'w = X'_{N+1}$  gives a system of equations as in (7.30). Here we have

$$\begin{pmatrix} V & -X \\ X' & 0 \end{pmatrix} \begin{pmatrix} w \\ \lambda \end{pmatrix} = \begin{pmatrix} s \\ X'_{N+1} \end{pmatrix}.$$

Next, we note a result using something called a Schur complement that identifies the inverse of a partitioned matrix. Let

$$M = \begin{pmatrix} A & B \\ C & D \end{pmatrix}$$

with  $A$  invertible. Then  $M$  has inverse

$$\begin{pmatrix} A^{-1} + A^{-1}B(M/A)^{-1}CA^{-1} & -A^{-1}B(M/A)^{-1} \\ -(M/A)^{-1}CA^{-1} & (M/A)^{-1} \end{pmatrix} \quad (7.39)$$

where

$$(M/A) = D - CA^{-1}B \quad (7.40)$$

is called the *Schur complement* of  $M$  with respect to  $A$ . We leave the verification to the reader.

Applying this result to the system of equations at hand gives that

$$w^* = (V^{-1} - V^{-1}X(X'V^{-1}X)X'V^{-1})s + (V^{-1}X(X'V^{-1}X)^{-1})X'_{N+1}$$

minimizes the prediction error variance and yields an unbiased estimator.

For this  $w^*$  we have

$$\begin{aligned} \hat{y}_{N+1} &= w^*Y \\ &= s'(V^{-1} - V^{-1}X(X'V^{-1}X)X'V^{-1})Y + X_{N+1}((X'V^{-1}X)^{-1}X'V^{-1})Y. \end{aligned}$$

Recalling that

$$\hat{\beta} = (X'V^{-1}X)^{-1}X'V^{-1}Y,$$

this reduces rather nicely to

$$\begin{aligned} s'V^{-1}Y - V^{-1}X\hat{\beta} + X_{N+1}\hat{\beta} \\ X_{N+1}\hat{\beta} + s'V^{-1}(Y - X\hat{\beta}). \end{aligned}$$

Notice that  $V$  and  $s$  are not specified by the data necessarily. As a matter of practice, these parameters are usually modeled with a small set of parameters themselves.

As a final example using a quadratic objective function, we consider the problem of determining a portfolio with minimum variance whose CAPM  $\beta$  is one. We shall explore this topic in much further depth in subsequent work when we introduce mean-variance optimization more thoroughly.

**Example 7.5.3.** We know that for a vector of portfolio weights,  $w \in \mathbb{R}^N$  and random return vector  $r \in \mathbb{R}^N$ , the variance of the portfolio is given by

$$\text{Var}(w'r) = w'\Sigma w$$

where  $\Sigma = \text{Cov}(r)$ .

We may identify the portfolio with smallest variance whose CAPM  $\beta$  is one by solving

$$\min_w \frac{1}{2} w' \Sigma w \\ \beta' w = 1$$

where  $\beta \in \mathbb{R}^N$  is the vector of individual asset  $\beta$ 's; i.e.,  $\beta_i$  is the CAPM  $\beta$  of asset  $i$ . We leave to the reader to prove that the portfolio  $\beta$  is in fact the weighted sum of asset  $\beta$ 's.

The solution to the optimization problem is readily obtained by considering the Lagrangian,

$$\mathcal{L}(w, \lambda) = \frac{1}{2} w' \Sigma w - \lambda (\beta' w - 1)$$

with gradient in  $w$

$$\nabla_w \mathcal{L}(w, \lambda) = \Sigma w - \lambda \beta.$$

The optimal  $w^*$  satisfies  $\nabla_w \mathcal{L}^* = 0$ , giving

$$w^* = \lambda \Sigma^{-1} \beta.$$

For  $w^*$  to be feasible, we require  $\beta' w^* = 1$ , so that

$$\lambda^* \beta' \Sigma^{-1} \beta = 1 \\ \lambda^* = (\beta' \Sigma^{-1} \beta)^{-1}.$$

Putting this all together,

$$w^* = \frac{\Sigma^{-1} \beta}{\beta' \Sigma^{-1} \beta}.$$

The weight  $w^*$  may be identified further. Let the weight used to construct the market returns be  $w_m$  and suppose that the returns in question,  $r$ , contain all components of the market index.

We know that for each asset,  $r_i$ ,  $\beta_i$  is given by

$$\beta_i = \frac{\text{Cov}(r_i, r_m)}{\text{Var}(r_m)},$$

The covariance between any particular asset return,  $r_i$ , and the market return  $r_m = w_m' r$  is given, using  $e_i$  as the vector with a one in the  $i$ th component and zeros elsewhere, by

$$\begin{aligned} \text{Cov}(r_i, r_m) &= \text{Cov}(e_i' r, w_m' r) \\ &= e_i' \Sigma w_m \\ &= \Sigma_i w_m \end{aligned}$$

where, necessarily,  $\Sigma_i$  is the  $i$ th row of the covariance matrix,  $\Sigma$ . The variance of the market return is clearly given by  $w'_m\Sigma w_m$ . As a result,

$$\beta = \frac{\Sigma w_m}{w'_m\Sigma w_m}.$$

Returning to  $w^*$ , we have

$$\begin{aligned}w^* &= \frac{\Sigma^{-1}\beta}{\beta'\Sigma^{-1}\beta} \\&= \frac{\Sigma^{-1}\Sigma w_m}{w'_m\Sigma^{-1}\Sigma w_m} \frac{1}{w'_m\Sigma w_m} (w'_m\Sigma w_m)^2 \\&= w_m.\end{aligned}$$

So that the minimum variance portfolio with unit  $\beta$  is the market portfolio.

Finally, we develop a linear programming problem related to regression. Here, we introduce the tilting function,  $\rho_\tau(\cdot)$ , parameterized in  $\tau$ . We will see that this function is related to identifying the sample  $\tau$  quantile of a distribution and will use this fact to develop what is called quantile regression. In both the case of quantile value estimation and quantile regression, we will show that the problem may be written as a linear programming problem in standard form.

**Example 7.5.4.** Given the sample  $\{y_t\}_{t=1}^N$ , the sample  $\tau$  quantile may be obtained by solving

$$\min_q \sum_{t=1}^N \rho_\tau(y_t - q) \quad (7.41)$$

where  $\rho_\tau$  is the tilting function defined by

$$\rho_\tau(x) = \tau \max(x, 0) + (1 - \tau) \max(-x, 0). \quad (7.42)$$

Equivalently,

$$\rho_\tau(x) = \begin{cases} \tau x & \text{if } x \ge 0 \\ (\tau - 1)x & \text{if } x < 0. \end{cases}$$

Notice that  $\rho_\tau$  takes nonnegative values.

We justify that the sample quantile,  $\hat{q}_\tau$ , may be obtained via  $\rho_\tau$ , through a proof using the population quantile,  $q_\tau = F^{-1}(\tau)$ , where  $F$  is the cumulative distribution function of the random variable  $Y$ .

For any real valued  $u$ , we have that

$$\begin{aligned}\mathbb{E}(\rho_\tau(Y-u)) &= \int_{-\infty}^{\infty} \rho_\tau(y-u)dF(y) \\&= \int_{-\infty}^{u} (\tau-1)(y-u)dF(y) + \int_{u}^{\infty} \tau y dF(y) \\&= (\tau-1) \int_{-\infty}^{u} y dF(y) \\&\quad - (\tau-1)u \int_{-\infty}^{u} dF(y) \\&\quad + \tau \int_{u}^{\infty} y dF(y) \\&\quad - \tau u \int_{u}^{\infty} dF(y).\end{aligned}$$

This expectation may be written as a function of  $u$  parameterized by  $\tau$  as  $L_\tau(u)$ . We are interested in minimizing this expected loss function and so take the first derivative in  $u$ . Doing so, we see that

$$L'_\tau(u) = (1-\tau) \int_{-\infty}^{u} dF(y) - \tau \int_{u}^{\infty} dF(y),$$

which may further be simplified as

$$L'_\tau(u) = (1-\tau)F(u) - \tau(1-F(u)) = F(u) - \tau,$$

and hence the function attains a minimum at  $u = F^{-1}(\tau)$  as originally claimed.

Having established the quantile property of  $\rho_\tau$ , we next write (7.41) as an LP problem. For

$$\rho_\tau(y_t - u) = \tau(y_t - u)_+ + (1 - \tau)(u - y_t)_+,$$

we construct variables,  $z_{t,+}$  and  $z_{t,-}$  such that each variable is nonnegative, and in addition,

$$\begin{aligned}z_{t,+} &\ge y_t - u \\z_{t,-} &\ge u - y_t.\end{aligned}$$

In these new variables, we may write (7.41) as

$$\begin{array}{rl}\min_{u, z_+, z_-} & \sum_{t=1}^{N} \tau z_{t,+} + (1 - \tau) z_{t,-} \\& \begin{array}{l}z_+ + u \ge Y \\z_- - u \ge -Y \\z_+ \ge 0 \\z_- \ge 0,\end{array}\end{array}\tag{7.43}$$

![Scatter plot showing Log Spread to Maturity (Y-axis, 0 to 6) versus Log Volatility (X-axis, -2 to 3.5). The data points are scattered, showing a positive relationship. Four quantile regression lines are plotted, ranging from bottom to top: 10%, 25%, 50%, 75%, and 90%. The lines show increasing slope as the quantile level increases, indicating a steeper relationship for higher quantiles.](1915a058b4bbae76204ce8141aa3fe35_img.jpg)

**Log Spread to Maturity for High Yield Bonds to Log Volatility**

Scatter plot showing Log Spread to Maturity (Y-axis, 0 to 6) versus Log Volatility (X-axis, -2 to 3.5). The data points are scattered, showing a positive relationship. Four quantile regression lines are plotted, ranging from bottom to top: 10%, 25%, 50%, 75%, and 90%. The lines show increasing slope as the quantile level increases, indicating a steeper relationship for higher quantiles.

Figure 7.2: Plot of various quantile regression levels when regressing log spreads on log realized equity volatility and its square, including an intercept term. Quantile levels range from (bottom to top): 10%, 25%, 50%, 75%, and 90%.

where  $Y = (y_1, \dots, y_N)'$ . From here, the additional variables and constraints needed to put the problem in standard form should be clear. We note, too, that these particular constraints may be refined a bit more by simply requiring requiring to  $z_+ - z_- = y - u$ . We have left the framework above for clarity of exposition, but encourage the interested reader to reframe the problem in this manner.

Based on the above, we define quantile regression as

$$\min_{\beta} \sum_{t=1}^{N} \rho_{\tau}(y_t - \beta' x_t), \quad (7.44)$$

in a manner similar to (4.11). The only difference from previous work in terms of the optimization problem that is implied being the change of loss function from the  $L^2$  norm to  $\rho_{\tau}(\cdot)$ . As this new problem is linear in  $\beta$ , we may write (7.44) as an LP problem in standard form as in (7.41). We note that an intercept term is required.

An example of the results from a single input variable are given in Figure 7.5.4. Here we have used log volatility and its square to estimate quantile regressors

for various quantiles. The boundary curves represent the 0.10 and 0.90 quantile regressions (from bottom to top). Notice that the shape of the regressions for these values differ slightly from what might be expected in a confidence interval curve from ordinary least squares as given by (4.34).

**Example 7.5.5.** Using the same framework as the previous example, we next consider the Huber loss based regression,

$$\min_{\beta}\sum_{t=1}^{N}\rho_{C}(y_{t}-\beta'x_{t}) \tag{7.45}$$

for some set of observations  $\{x_t\}_{t=1}^N$ , and with  $\rho_C(\cdot)$  the Huber loss with parameter  $C$ , defined by

$$\rho_{C}(x)=\begin{cases} |x|^2 & \text{for } |x|\le C \\ 2C|x|-C^2 & \text{for } |x|>C. \end{cases} \tag{7.46}$$

With a bit of examination, it becomes clear that the Huber loss considers square losses ( $L^2$ ) inside a band of width  $2C$ , and linear ( $L^1$ ) outside this band, with some care taken to assure continuity at the boundary. We proceed by formulating (7.45) as a constrained quadratic programming problem.

To do so, we again introduce auxiliary variables. In the present case, we seek variables,  $z_t$  and  $u_t$ , such that  $|y_t-\beta'x_t|=z_t+u_t$  and  $z_t$  takes up that portion of the absolute value up to  $C$  and  $u_t$  the remainder. This designation is actually sufficient to proceed. Consider

$$\begin{array}{ll} \min_{z,u,\beta} & ||z||^2+2C1'u \\ & y_t-\beta'x_t\le z_t+u_t \text{ for } 1\le t\le N \\ & -z_t-u_t\le y_t-\beta'x_t \text{ for } 1\le t\le N \\ & 0\le z\le C \\ & 0\le u \end{array} \tag{7.47}$$

Noting the identification,

$$|y_t-\beta'x_t|=\min(|y_t-\beta'x_t|,C)+\max(|y_t-\beta'x_t|-C,0)$$

and that at the solution to (7.47), we will have equality in the first two conditions gives

$$|y_t-\beta'x_t|=z_t+u_t,$$

and the correspondence between (7.45) and (7.47) follows.

Clearly as  $C\rightarrow\infty$ , the above loss function tends towards the standard least squares problem considered extensively already. For intermediate,  $C$ , however, we see that the Huber loss based regression reduces the overall impact (and hence, the fitting to) outliers in  $\{y_t\}_t$  as determined by  $C$ . The question of how to determine such  $C$  is left to a subsequent chapter.

### Exercises

1. We say that a set  $C$  is a cone if for  $\alpha > 0$ ,  $\alpha c \in C$  for every  $c \in C$ . Prove that  $\mathcal{F}(x)$  as in (7.9) is a cone.
2. Prove that if  $x^*$  is a regular point, with  $(x^*, \lambda^*)$  satisfying (7.11), then  $\lambda^*$  is unique.
3. Let  $(x^*, \lambda^*)$  satisfy (7.11). Show that  $\mathcal{L}(x^*, \lambda^*) = f^*$ .
4. Prove that the set

$$K = \{x | c_i(x) \ge k_i\}$$

is convex when each  $c_i$  is concave and leave the proof to the reader.

1. Verify (7.39), that a partitioned matrix

$$\begin{pmatrix} A & B \\ C & D \end{pmatrix}$$

with  $A$  invertible has inverse

$$\begin{pmatrix} A^{-1} + A^{-1}B(M/A)^{-1}CA^{-1} & -A^{-1}B(M/A)^{-1} \\ -(M/A)^{-1}CA^{-1} & (M/A)^{-1} \end{pmatrix}$$

where

$$(M/A) = D - CA^{-1}B.$$

1. Rigorously prove that each optimal Lagrange multiplier in the active set is exactly the sensitivity of the objective function value at the optimal solution when a small change in the constraint value is made. That is, modify the proof shown in the text by using little- $o$  notation as in (5.17).
2. What is the impact of a nonzero Lagrange multiplier when the objective function is multiplied by a scalar  $c > 0$ ? What is the impact of multiplying a single constraint function,  $c_i(x)$  by  $c$  (leaving the objective function fixed)? (This result coupled with the previous exercise shows that while Lagrange multipliers may be viewed as sensitivities, they are also scale dependent; i.e., they are not necessarily unitless.)
3. Let  $f(x) \in \mathbb{C}^2$ .
   1. Using a degree two Taylor expansion, write down a quadratic approximation  $f_q(x + \delta) = h + g'\delta + \frac{1}{2}\delta'G\delta$  in terms of  $f(x)$ , the gradient  $\nabla f(x)$ , and the Hessian,  $\nabla^2 f(x)$ .
   2. Let  $q(\delta) = f_q(x + \delta)$ . Assuming the matrix  $G$  you found above is always positive definite, find an equation for  $\delta^*$ , the minimizer of  $q(\delta)$ .
   3. How does your  $\delta^*$  above relate to the Newton step we derived in Newton's method?

(d) Next, we generalize the work we did to obtain what we called the  $K$ -matrix in a quadratic programming problem with equality constraints. Consider

$$\begin{array}{ll}\min_x & f(x) \\ \text{subject to} & Ax = b\end{array}$$

1. Let  $x^k$  be a feasible point. Define  $x^{k+1} = x^k + \delta$  for some  $\delta$ . Write down a condition that must hold for  $\delta$  for  $x^{k+1}$  to remain feasible.
2. Let  $q_k(\delta) = f_q(x^k + \delta)$ , where  $f_q(x^k + \delta)$  is the approximation of  $f(x^k + \delta)$  using a second degree Taylor expansion as earlier in the exam. Consider the approximate problem

$$\begin{array}{ll}\min_{\delta} & q_k(\delta) \\ \text{subject to} & \tilde{A}\delta = \tilde{b}\end{array}$$

where  $\tilde{A}$  and  $\tilde{b}$  are the conditions needed to keep  $x^{k+1}$  feasible as determined above. Write down the Lagrangian for this new problem in terms of  $f(x^k)$ , the gradient  $\nabla f(x^k)$ , the Hessian,  $\nabla^2 f(x^k)$ ,  $\tilde{A}$ , and  $\tilde{b}$ .

1. Write down the KKT conditions for the new problem in  $\delta$ .
2. Combine the above conditions into a matrix equation in a similar manner as we did for the equality constrained quadratic programming problem. You are looking for a matrix  $K$  and a vector  $\kappa$  such that

$$K \begin{pmatrix} \delta \\ \lambda \end{pmatrix} = \kappa$$

where  $\lambda$  is the vector of Lagrange multipliers.

1. Write (7.32) in standard form.
2. Write (7.44) in standard form.
3. Complete the proof of the Strong Duality Theorem by showing that if the primal (dual) is unbounded, then the dual (primal) is infeasible.
4. Let  $f(x) = \sum_{i=1}^N x_i \ln x_i$  for  $x \in \mathbb{R}^N$ , and  $x \ge 0$ .
   1. What is  $\nabla f(x)$ ?
   2. Consider the problem

$$\begin{array}{ll}\min_x & f(x) \\ \text{subject to} & \frac{1}{N} \sum_{i=1}^N x_i = \mu\end{array}$$

1. Write down the KKT conditions for this problem.
2. Using your KKT equations, solve for  $x_i$  in terms of  $\lambda$ , the Lagrange multiplier used in the KKT condition.

13. Convince yourself that the condition that  $x$  be a regular point coincides with the Jacobian of

$$F(x) = \begin{pmatrix} c_1(x) \\ \vdots \\ c_m(x) \end{pmatrix}$$

being invertible for  $\mathcal{A}(x) = \{1, \dots, m\}$ .